Pre Processing function

In [1]:
from nltk.stem import WordNetLemmatizer 
import nltk
nltk.download('wordnet')


remove_words = ["@handle", "RT", "http"]
lemmatizer = WordNetLemmatizer() 

def pre_process(sentence, max_length):
    sentence = sentence.split()
    target_remove = set()
    for i, token in enumerate(sentence):
        for target in remove_words:
            if (target == "http") and (target in token.lower()):
                target_remove.add(token)
                break
            if target in token:
                target_remove.add(token)
                break
    for target in target_remove:
        while target in sentence:
            sentence.remove(target)
            
    for i, token in enumerate(sentence):
        sentence[i] = lemmatizer.lemmatize(token)
        
    max_length = max(max_length, len(sentence))
    sentence = ' '.join(sentence)
    return sentence, max_length

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/hanxunhuang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Load Train data to train_dict. 
train_dict[id] = [[train_instace1], [train_instance2] ....]

In [2]:
import csv
import collections
import numpy as np

train_file_path = "data/train_tweets.txt"
train_dict = collections.defaultdict(list)
max_length = 0

length_array = []
with open(train_file_path, encoding='utf-8') as tsvfile:
    reader = tsvfile.readlines()
    for i, row in enumerate(reader):
        row = row.strip().split("\t")
        id = int(row[0])
        instance, max_length = pre_process(row[1], max_length)
        if not instance == "":
            train_dict[id].append(instance)
            length_array.append(len(instance.split()))
            if len(instance) == 0:
                print("Error")
    print("Total rows: %d" % i)
    
print("Total ids: %d" % len(train_dict))
print("Longest Sentence: %d" % (max_length))


Total rows: 328931
Total ids: 9295
Longest Sentence: 37


In [ ]:
print(train_dict[1319])

Split Dev Train
!! There are Some ID have 0 train instance

In [3]:
import random
import numpy as np

dev_split = 0.1
train_split = 1 - dev_split

dev_set_dict = {}
train_set_dict = {}

for id in train_dict:
    target_list = train_dict[id]
    length = len(target_list)
    random.shuffle(target_list)
    split = int(np.ceil(length*dev_split))
    dev_set_dict[id] = target_list[:split]
    train_set_dict[id] = target_list[split:length]
    
print(len(dev_set_dict), len(train_set_dict), len(train_dict))


9295 9295 9295


Build ID idx and train/dev set save to file

In [5]:
import pickle
import pandas as pd

def save_to_file(target_dict, file_path):
    id_list = []
    sentence_list = []
    for id in target_dict:
        for sentence in target_dict[id]:
            id_list.append(id)
            sentence_list.append(sentence)
      
    id_list = np.array(id_list)
    sentence_list = np.array(sentence_list)
    random_index = np.array(range(len(sentence_list)))
    random.shuffle(random_index)
    id_list = id_list[random_index]
    sentence_list = sentence_list[random_index]
    
    dataframe = pd.DataFrame({'id':id_list,'sentence':sentence_list})
    dataframe.to_csv(file_path,index=False,sep='\t',header=None)
    print(len(id_list))
    return

dev_set_path = 'data/v4/dev_set.txt'
train_set_path = 'data/v4/train_set.txt'
idx_file_path = 'data/v4/idx.pickle'

save_to_file(dev_set_dict, dev_set_path)
save_to_file(train_set_dict, train_set_path)


# Build IDX and Save
idx = {}
for i, id in enumerate(train_dict):
    idx[id] = i
print(len(idx))

with open(idx_file_path, 'wb') as handle:
    pickle.dump(idx, handle, protocol=pickle.HIGHEST_PROTOCOL)


36116
291758
9295


In [ ]:
print(idx)

In [ ]:
from feature_extract_inference import FeatureExtract
test_model = FeatureExtract(checkpoints_path="/Users/hanxunhuang/Desktop/checkpoints/coconut_extract_model_v2_best.pth")

In [ ]:
from numpy import dot
from numpy.linalg import norm

feature = test_model.get_features(["Shannon was right, Hamilton should step down #fb", "Listen live on as I debate @handle Friday Oct 2nd at 10:00 am #fb"])
feature1 = feature[0]
feature2 = feature[1]

cos_sim = dot(feature1, feature2)/(norm(feature1)*norm(feature2))
print(cos_sim)

